# Supervised classification

In the data.csv there are letters (uppercases and lowercases) and numbers, 28x28 pixels in a row format.

* First, you need to know which labels are which, meaning you need to visualize some data to realize which number labels represents a letter, or a number.

In [10]:
import requests
from io import BytesIO
from zipfile import ZipFile

In [16]:
url="https://github.com/eduardoheros/lab-supervised-classification/blob/master/data_all.zip?raw=true"
url_file=requests.get(url)

In [19]:
zipfile=ZipFile(BytesIO(url_file.content))

In [25]:
data=zipfile.extractall()

In [26]:
data

* Now, try to train a classifier model to predict the uppercases. Use every single model you know for classification.

In [ ]:
# Your code here

* Try to do the same thing with lowercases.

In [ ]:
# Your code here

* Try to do the same thing with numbers.

In [ ]:
# Your code here